In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import dowhy
import shap
import warnings
from dowhy import CausalModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import mlflow

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)


print(mlflow.__version__)

/Users/yyy/Library/CloudStorage/OneDrive-Personal/MMA/2024WINTER1/INSY695/Spotify-Music/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.12.1


In [2]:
df = pd.read_csv('../data/raw/training_data_original.csv')

In [3]:
df.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.748,0.916,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.726,0.815,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.675,0.931,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.718,0.930,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.650,0.833,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


### Check for duplicate tracks and remove the lowest popularity song

In [4]:
df = df.sort_values('track_popularity', ascending=False).drop_duplicates('track_name').sort_index()

# Only keep songs wiht popularity > 0
df = df[df['track_popularity'] > 0]

### Convert time to minutes from milliseconds 

In [5]:
# Convert duration_ms to minutes with 2 decimal places
df['duration_mins'] = df['duration_ms'] / 60000
df['duration_mins'] = df['duration_mins'].round(2)

#drop duration_ms
df = df.drop(columns=['duration_ms'])

In [6]:
df.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.748,0.916,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,3.25
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.675,0.931,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,2.94
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.718,0.930,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,2.82
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.650,0.833,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,3.15
6,6b1RNvAcJjQH73eZO4BLAB,Post Malone (feat. RANI) - GATTÜSO Remix,Sam Feldt,69,6703SRPsLkS4bPtMFFJes1,Post Malone (feat. RANI) [GATTÜSO Remix],2019-08-29,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.542,0.903,4,-2.419,0,0.0434,0.0335,0.000005,0.1110,0.367,127.936,3.46


### Create new column `release_year` to capture the year of the song release

In [7]:
df['track_album_release_date'] = pd.to_datetime(df['track_album_release_date'], errors='coerce')
df['release_year'] = df['track_album_release_date'].dt.year

### Remove unwanted columns

In [8]:
df = df.drop(columns=['track_id', 'track_name', 'track_album_id', 'track_album_name', 'track_album_release_date', 'playlist_name', 'playlist_id'])

### Causal model building

In [9]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment('Causal_Analysis_Experiment')

with mlflow.start_run():
    mlflow.autolog()

    causal_model = CausalModel(
        data=df,
        treatment=['duration_mins'],
        outcome='track_popularity',
        common_causes=df.drop(columns=['track_artist','playlist_genre','playlist_subgenre'  ,'duration_mins', 'track_popularity']).columns.tolist(),
        effect_modifiers=['release_year']
    )

    mlflow.log_param("model", causal_model)

    # causal_model.view_model(file_name="../reports/figures/causal_model")

    identified_effect = causal_model.identify_effect()
    mlflow.log_param("identified_effect", identified_effect)

    causal_estimate = causal_model.estimate_effect(identified_effect, method_name="backdoor.linear_regression", target_units="ate")
    mlflow.log_param("causal_estimate", causal_estimate.value)

    estimate = causal_model.estimate_effect(identified_effect, method_name="backdoor.linear_regression", test_significance=True)
    mlflow.log_param("estimate", estimate.value)

    refutation_results_rcc = causal_model.refute_estimate(
        identified_effect,
        causal_estimate,
        method_name="random_common_cause",
        test_significance=True,
    )

    mlflow.log_param("refutation_result_rcc", refutation_results_rcc.refutation_result)

    refutation_results_subset = causal_model.refute_estimate(
        identified_effect,
        causal_estimate,
        method_name="data_subset_refuter",
        subset_fraction=0.9,
        test_significance=True,
    )
    mlflow.log_param("refutation_result_subset", refutation_results_subset.refutation_result)

    refutation_results_bootstrap = causal_model.refute_estimate(
        identified_effect,
        causal_estimate,
        method_name="bootstrap_refuter",
        num_simulations=100,
        test_significance=True,
    )
    mlflow.log_param("refutation_result_bootstrap", refutation_results_bootstrap.refutation_result)

mlflow.end_run()

2024/04/25 16:14:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/04/25 16:14:20 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of statsmodels. If you encounter errors during autologging, try upgrading / downgrading statsmodels to a supported version, or try upgrading MLflow.
2024/04/25 16:14:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
